## Sequence Model

recurrent neural network는 어떠한 형태의 state를 가지는 네트워크입니다. 예를 들어, 출력이 다음 입력의 일부로 사용될 수 있으므로 네트워크가 시퀀스를 통과 할 떄 정보가 전파 될 수 있습니다. LSTM의 경우, 시퀀스의 각 요소에 대해 대응하는 숨은 상태 $h_{t}$가 있으며, 원칙적으로 시퀀스 앞부분의 임의의 점으로 부터 정보를 포함 할 수 있습니다. Hidden state를 사용하여 다양한 것을 할 수 있습니다.

### LSTM in pytorch

Pytorch 의 LSTM은 모든 입력이 3D tensor가 될 것으로 기대합니다. 첫번째 축은 sequence 자체이고, 두번쨰 축은 mini_batch의 instance를 인덱싱하고, 세번째 축은 input요소를 인덱싱합니다.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
# input 3차원, output도 3ckdnjs
lstm = nn.LSTM(3,3)
# length 5짜리 sequence를 만든다.
inputs = [torch.randn(1,3) for _ in range(5)]

In [3]:
inputs

[tensor([[-0.5525,  0.6355, -0.3968]]),
 tensor([[-0.6571, -1.6428,  0.9803]]),
 tensor([[-0.0421, -0.8206,  0.3133]]),
 tensor([[-1.1352,  0.3773, -0.2824]]),
 tensor([[-2.5667, -1.4303,  0.5009]])]

In [4]:
hidden = (torch.randn(1,1,3),
          torch.randn(1,1,3))

for i in inputs:
    out, hidden = lstm(i.view(1,1,-1), hidden)

In [5]:
out

tensor([[[-0.3600,  0.0893,  0.0215]]], grad_fn=<CatBackward>)

In [6]:
hidden

(tensor([[[-0.3600,  0.0893,  0.0215]]], grad_fn=<ViewBackward>),
 tensor([[[-1.1298,  0.4467,  0.0254]]], grad_fn=<ViewBackward>))

In [7]:
inputs = torch.cat(inputs).view(len(inputs),1,-1)
hidden = (torch.randn(1,1,3), torch.randn(1,1,3))
out, hidden = lstm(inputs, hidden)

print(out)
print(hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<CatBackward>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<ViewBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<ViewBackward>))


In [8]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [9]:
train_data = [
    ("The dog ate the apple".split(), ["DET","NN","V","DET","NN"]),
    ("Everybody read that book".split(), ["NN","V","DET","NN"])
]

word_to_ix = {}
for sent, tags in train_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [10]:
tag_to_ix = {"DET":0, "NN":1, "V":2}
Embeddind_dim = 6
Hidden_dim = 6

In [11]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM은 input으로 word embedding을 ouput의 hidden states를 받는다.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        # The Linear layer 는ㄴ hidden state space에서 tag space로 mapping 해줍니다.
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return (torch.zeros(1,1,self.hidden_dim),
               torch.zeros(1,1,self.hidden_dim))
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence),1,-1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [12]:
train_data

[(['The', 'dog', 'ate', 'the', 'apple'], ['DET', 'NN', 'V', 'DET', 'NN']),
 (['Everybody', 'read', 'that', 'book'], ['NN', 'V', 'DET', 'NN'])]

In [13]:
model = LSTMTagger(Embeddind_dim, Hidden_dim, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)


In [14]:
# traing 전의 socre를 확인하고, element 
with torch.no_grad():
    inputs = prepare_sequence(train_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)
    
for epoch in range(300):
    for sentence, tags in train_data:
        # step 1, Remeber that pytorch accumulates gradients
        model.zero_grad()
        # LSTM의 hidden state
        model.hidden = model.init_hidden()
        
        # step 2, get input networds 에 넣어주자
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        # step3, run our forward pass
        tag_scores = model(sentence_in)
        
        # step4, compute the loss, gradients, and update the parameter by calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        
with torch.no_grad():
    inputs = prepare_sequence(train_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)
        

tensor([[-1.1389, -1.2024, -0.9693],
        [-1.1065, -1.2200, -0.9834],
        [-1.1286, -1.2093, -0.9726],
        [-1.1190, -1.1960, -0.9916],
        [-1.0137, -1.2642, -1.0366]])
tensor([[-0.0858, -2.9355, -3.5374],
        [-5.2313, -0.0234, -4.0314],
        [-3.9098, -4.1279, -0.0368],
        [-0.0187, -4.7809, -4.5960],
        [-5.8170, -0.0183, -4.1879]])
